In [ ]:
import pymysql
import pandas as pd
import json
from datetime import datetime
import os

# مسیرها
OUTPUT_DIR = r"C:\BI\KZCCPP\G11\Lube Oil System"
OUTPUT_FILE = "lube_oil_system_data_g11.xlsx"
FULL_PATH = os.path.join(OUTPUT_DIR, OUTPUT_FILE)
LOG_FILE = os.path.join(OUTPUT_DIR, "lube_oil_export_log.txt")

os.makedirs(OUTPUT_DIR, exist_ok=True)

def log(msg):
    print(msg)
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}\n")

def get_connection():
    return pymysql.connect(
        host="127.0.0.1",
        port=3306,
        user="root",
        password="",        # پسورد رو اینجا بنویس اگر داری
        database="dsas",
        charset="utf8mb4",
        cursorclass=pymysql.cursors.DictCursor,
        autocommit=True
    )

log("شروع استخراج داده‌ها از جدول main_table_mhi_lube_oil_11 — با RecordDate و RecordTime")

try:
    conn = get_connection()
    cursor = conn.cursor()

    # کوئری اصلی — اضافه شدن RecordDate و RecordTime
    query = """
    SELECT 
        `id`,
        `RecordDate`,
        `RecordTime`,
        `AssetID_8341`,
        `AssetID_8342`,
        `AssetID_8343`,
        `AssetID_8344`,
        `AssetID_8346`,
        `AssetID_9286`,
        `AssetID_9287`
    FROM `main_table_mhi_lube_oil_11`
    ORDER BY `id` ASC
    """
    
    log("در حال اجرای کوئری...")
    cursor.execute(query)
    rows = cursor.fetchall()
    
    total_rows = len(rows)
    log(f"تعداد ردیف‌های دریافت‌شده از دیتابیس: {total_rows:,}")

    if total_rows == 0:
        log("جدول خالی است!")
        pd.DataFrame({"پیام": ["جدول هیچ ردیفی ندارد"]}).to_excel(FULL_PATH, index=False)
    else:
        data = []
        for row in rows:
            clean_row = {}
            clean_row['id'] = row['id']
            clean_row['RecordDate'] = row['RecordDate']  # تاریخ
            clean_row['RecordTime'] = row['RecordTime']  # زمان

            # پردازش ستون‌های AssetID (مثل قبل)
            for col in ['AssetID_8341','AssetID_8342','AssetID_8343','AssetID_8344',
                        'AssetID_8346','AssetID_9286','AssetID_9287']:
                value = row[col]
                if value is None:
                    clean_row[col] = None
                elif isinstance(value, (bytes, bytearray)):
                    try:
                        clean_row[col] = value.decode('utf-8')
                    except:
                        clean_row[col] = str(value)
                else:
                    try:
                        parsed = json.loads(value)
                        clean_row[col] = json.dumps(parsed, ensure_ascii=False, indent=2)
                    except (json.JSONDecodeError, TypeError):
                        clean_row[col] = str(value)
            
            data.append(clean_row)

        # ساخت دیتافریم
        df = pd.DataFrame(data)

        # ترتیب دلخواه ستون‌ها در خروجی اکسل
        desired_columns = [
            'id',
            'RecordDate',      # اول تاریخ
            'RecordTime',      # بعد زمان
            'AssetID_8341',
            'AssetID_8342',
            'AssetID_8343',
            'AssetID_8344',
            'AssetID_8346',
            'AssetID_9286',
            'AssetID_9287'
        ]

        # اگر همه ستون‌ها موجود باشن، ترتیب رو اعمال کن
        df = df[desired_columns]

        # ذخیره در اکسل
        df.to_excel(FULL_PATH, index=False, engine='openpyxl')
        
        log(f"موفقیت‌آمیز! {total_rows:,} ردیف با RecordDate و RecordTime ذخیره شد.")
        log(f"فایل: {FULL_PATH}")
        log("نمونه ۲ ردیف اول:")
        log(df.head(2).to_string())

except Exception as e:
    log(f"خطا: {e}")
    import traceback
    log(traceback.format_exc())

finally:
    try:
        cursor.close()
        conn.close()
    except:
        pass

log("پایان فرآیند — خروجی شامل RecordDate و RecordTime است.")

شروع استخراج داده‌ها از جدول main_table_mhi_lube_oil_11 — با RecordDate و RecordTime
در حال اجرای کوئری...
تعداد ردیف‌های دریافت‌شده از دیتابیس: 10,928
موفقیت‌آمیز! 10,928 ردیف با RecordDate و RecordTime ذخیره شد.
فایل: C:\BI\lube_oil_system_data_g11.xlsx
نمونه ۲ ردیف اول:
     id  RecordDate RecordTime AssetID_8341 AssetID_8342 AssetID_8343 AssetID_8344 AssetID_8346 AssetID_9286 AssetID_9287
0  5996  2023-12-03   04:24:53         0.07         12.1         69.0       -240.0          6.9          7.2         1.28
1  5997  2023-12-03   00:27:03         0.06         12.1         69.0       -240.0          6.9          7.2         1.28
پایان فرآیند — خروجی شامل RecordDate و RecordTime است.
